In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import mysql.connector
from mysql.connector import Error
import csv


C:\Users\syeda.fatima\AppData\Local\Temp\ipykernel_28508\3736968601.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def load_scraped_data_to_mysql(link="https://www.geeksforgeeks.org/blood-group-chart"):
    soup = BeautifulSoup(requests.get(link).text)
    # first we should find our table object:
    table = soup.find('table')
    # then we can iterate through each row and extract either header or row values:
    header = []
    rows = []
    for i, row in enumerate(table.find_all('tr')):
        updated_rows =[]
        if i == 0:
            header = [el.text.strip() for el in row.find_all('th')]
        else:
            updated_rows.extend([el.text.strip() for el in row.find_all('th')])
            updated_rows.extend([el.text.strip() for el in row.find_all('td')])
            # print(updated_rows)
            rows.append(updated_rows)

    df = pd.DataFrame(rows)
    df.columns = header
    # print(df)
    csv_file_path = 'BloodGroupCompatability.csv'
    df.to_csv(csv_file_path, index= False)
    csv_to_sql(csv_file_path, 'BloodGroupCompatibility')

def connect_to_database():
    """Establish a connection to the MySQL database."""
    try:
        connection = mysql.connector.connect(host='CSSQL',
                                             database='mm_team02_02',
                                             user='mm_team02_02',
                                             password='mm_team02_02Pass-')
        return connection
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
        return None

def csv_to_sql(csv_file_path, table_name):
    connection = connect_to_database()
    if connection is not None:
        cursor = connection.cursor()
        try:
            drop_table_stmt = f"DROP TABLE IF EXISTS `{table_name}`"
            cursor.execute(drop_table_stmt)

            with open(csv_file_path, mode='r', encoding='utf-8') as csvfile:
                csvreader = csv.reader(csvfile)
                headers = next(csvreader)
                column_definitions = ', '.join([f"`{header.strip()}` VARCHAR(30)" for header in headers])
                
                create_table_stmt = f"""
                CREATE TABLE IF NOT EXISTS `{table_name}` (
                    {column_definitions}
                )
                """
                cursor.execute(create_table_stmt)
                
                columns = ', '.join([f"`{header.strip()}`" for header in headers])
                placeholders = ', '.join(['%s'] * len(headers))
                insert_stmt = f"INSERT INTO `{table_name}` ({columns}) VALUES ({placeholders})"
                
                for row in csvreader:
                    if len(row) == len(headers): 
                        cursor.execute(insert_stmt, row)
                    else:
                        print(f"Row has a different number of elements than expected: {row}")
                
                connection.commit()
                print(f"CSV data has been successfully inserted into the `{table_name}` table.")
        except Error as e:
            print(f"Error while inserting data into MySQL: {e}")
            connection.rollback()  
        finally:
            cursor.close()
            if connection.is_connected():
                connection.close()
    else:
        print("Failed to connect to the database")

load_scraped_data_to_mysql()

CSV data has been successfully inserted into the `BloodGroupCompatibility` table.
